## Importing libraries

In [ ]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

# Math
import math

# HuggingFace libraries 
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

# Pathlib 
from pathlib import Path

# typing
from typing import Any

# Library for progress bars in loops
from tqdm import tqdm

# Importing library of warnings
import warnings

implementing transformer architecture
<p align="center">
<img height="400" src="../image.png" >  
</p>
<p align="center">
Picture：Transformer architecture
</p> 

In [ ]:
# 創建輸入嵌入層
class InputEmbeddings(nn.Module):
    
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model # 向量的維度 (512)
        self.vocab_size = vocab_size # 詞彙表的大小
        self.embedding = nn.Embedding(vocab_size, d_model) # PyTorch 層，用於將整數索引轉換為稠密嵌入
        
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model) # 正規化嵌入向量的方差

In [ ]:
# 創建位置編碼層
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # 模型的維度
        self.seq_len = seq_len # 最大序列長度
        self.dropout = nn.Dropout(dropout) # Dropout層，用於防止過擬合
        
        # 創建一個形狀為 (seq_len, d_model) 的位置編碼矩陣，初始值為零
        pe = torch.zeros(seq_len, d_model)
        
        # 創建一個表示位置的張量 (0 到 seq_len - 1)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # 將 'position' 轉換為 2D 張量 [seq_len, 1]
        
        # 創建位置編碼公式中的分母項
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # 對 pe 的偶數索引應用正弦函數
        pe[:, 0::2] = torch.sin(position * div_term)
        # 對 pe 的奇數索引應用餘弦函數
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # 在 pe 矩陣的開頭增加一個維度，以便處理批量數據
        pe = pe.unsqueeze(0)
        
        # 將 'pe' 註冊為緩衝區，緩衝區是一個不被認為是模型參數的張量
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # 將位置編碼加到輸入張量 x 上
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x) # 使用 Dropout 進行正則化


In [ ]:
# 創建層正規化
class LayerNormalization(nn.Module):
    
    def __init__(self, eps: float = 10**-6) -> None: # 我們將 epsilon 定義為 0.000001 以避免除以零
        super().__init__()
        self.eps = eps
        
        # 我們將 alpha 定義為可訓練參數並將其初始化為全 1
        self.alpha = nn.Parameter(torch.ones(1)) # 一維張量，將用於縮放輸入數據
        
        # 我們將 bias 定義為可訓練參數並將其初始化為全 0
        self.bias = nn.Parameter(torch.zeros(1)) # 一維張量，將加到輸入數據上
        
    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True) # 計算輸入數據的均值，保持維度不變
        std = x.std(dim=-1, keepdim=True) # 計算輸入數據的標準差，保持維度不變
        
        # 返回正規化後的輸入數據
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


In [ ]:
# 創建前饋神經網絡層
class FeedForwardBlock(nn.Module):
    
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        # 第一個線性變換
        self.linear_1 = nn.Linear(d_model, d_ff) # W1 和 b1
        self.dropout = nn.Dropout(dropout) # Dropout 用於防止過擬合
        # 第二個線性變換
        self.linear_2 = nn.Linear(d_ff, d_model) # W2 和 b2
        
    def forward(self, x):
        # (批次大小, 序列長度, d_model) --> (批次大小, 序列長度, d_ff) --> (批次大小, 序列長度, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x)))) # 使用 ReLU 激活函數和 Dropout 進行正則化


In [ ]:
# 創建多頭注意力模塊
class MultiHeadAttentionBlock(nn.Module):
    
    def __init__(self, d_model: int, h: int, dropout: float) -> None: # h = 頭的數量
        super().__init__()
        self.d_model = d_model
        self.h = h
        
        # 確保模型的維度可以被頭的數量整除
        assert d_model % h == 0, 'd_model 不能被 h 整除'
        
        # d_k 是每個注意力頭的 key, query, 和 value 向量的維度
        self.d_k = d_model // h # d_k 的公式，與 "Attention Is All You Need" 論文一致
        
        # 定義權重矩陣
        self.w_q = nn.Linear(d_model, d_model) # W_q
        self.w_k = nn.Linear(d_model, d_model) # W_k
        self.w_v = nn.Linear(d_model, d_model) # W_v
        self.w_o = nn.Linear(d_model, d_model) # W_o
        
        self.dropout = nn.Dropout(dropout) # Dropout 層，用於防止過擬合
        
    
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):# mask => 當我們希望某些單詞不與其他單詞互動時，我們 "隱藏" 它們
        
        d_k = query.shape[-1] # query, key 和 value 的最後一個維度
        
        # 我們按公式計算注意力 (Q,K,V) 
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k) # @ = PyTorch 中的矩陣乘法符號
        
        # 在應用 softmax 之前，我們應用 mask 來隱藏一些單詞之間的互動
        if mask is not None: # 如果定義了 mask ...
            attention_scores.masked_fill_(mask == 0, -1e9) # 將 mask 等於 0 的每個值替換為 -1e9
        attention_scores = attention_scores.softmax(dim=-1) # 應用 softmax
        if dropout is not None: # 如果定義了 dropout ...
            attention_scores = dropout(attention_scores) # 我們應用 dropout 來防止過擬合
            
        return (attention_scores @ value), attention_scores # 按公式將輸出矩陣乘以 V 矩陣
        
    def forward(self, q, k, v, mask): 
        
        query = self.w_q(q) # Q' 矩陣
        key = self.w_k(k) # K' 矩陣
        value = self.w_v(v) # V' 矩陣
        
        # 將結果分割成多個小矩陣以供不同的頭使用
        # 將嵌入 (第三維度) 分成 h 部分
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2) # 轉置 => 將頭移到第二維度
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2) # 轉置 => 將頭移到第二維度
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2) # 轉置 => 將頭移到第二維度
        
        # 獲得輸出和注意力分數
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # 獲得 H 矩陣
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        
        return self.w_o(x) # 將 H 矩陣乘以權重矩陣 W_o，得到多頭注意力矩陣


In [ ]:
# 創建殘差連接
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout) # 使用 Dropout 層來防止過擬合
        self.norm = LayerNormalization() # 使用正規化層 
    
    def forward(self, x, sublayer):
        # 將輸入正規化並將其加到原始輸入 'x' 上，這創建了殘差連接過程
        return x + self.dropout(sublayer(self.norm(x)))


In [ ]:
# 創建編碼器塊
class EncoderBlock(nn.Module):
    
    # 這個塊接收多頭注意力塊和前饋神經網絡塊，以及殘差連接的 dropout 率
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        # 存儲自注意力塊和前饋神經網絡塊
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)]) # 2 個帶有 dropout 的殘差連接
        
    def forward(self, x, src_mask):
        # 應用第一個帶有自注意力塊的殘差連接
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask)) # 三個 'x' 分別對應於 query, key 和 value 的輸入，加上源掩碼
        
        # 應用第二個帶有前饋神經網絡塊的殘差連接
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x # 應用自注意力和前饋層以及殘差連接後的輸出張量


In [ ]:
# 創建編碼器
# 編碼器可以包含多個編碼器塊
class Encoder(nn.Module):
    
    # 編碼器接收 'EncoderBlock' 的實例
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers # 存儲編碼器塊
        self.norm = LayerNormalization() # 用於正規化編碼器層輸出的正規化層
        
    def forward(self, x, mask):
        # 遍歷 self.layers 中存儲的每個編碼器塊
        for layer in self.layers:
            x = layer(x, mask) # 將每個編碼器塊應用於輸入張量 'x'
        return self.norm(x) # 正規化輸出


In [ ]:
# 創建解碼器塊
class DecoderBlock(nn.Module):
    
    # 解碼器塊接收兩個多頭注意力塊。一個是自注意力，另一個是交叉注意力。
    # 它還接收前饋神經網絡塊和 dropout 率
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)]) # 包含三個帶有 dropout 的殘差連接的列表
        
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        
        # 自注意力塊，使用查詢、鍵和值加上目標語言掩碼
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        
        # 交叉注意力塊，使用兩個 'encoder_output' 作為鍵和值加上源語言掩碼，同時使用 'x' 作為解碼器的查詢
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        
        # 帶有殘差連接的前饋塊
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x


In [ ]:
# 創建解碼器
# 解碼器可以包含多個解碼器塊
class Decoder(nn.Module):
    
    # 解碼器接收 'DecoderBlock' 的實例
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        
        # 存儲 'DecoderBlock'
        self.layers = layers
        self.norm = LayerNormalization() # 用於正規化輸出的層
        
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        
        # 遍歷 self.layers 中存儲的每個解碼器塊
        for layer in self.layers:
            # 將每個解碼器塊應用於輸入 'x'、編碼器輸出以及源和目標掩碼
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x) # 返回正規化後的輸出


In [ ]:
# 創建線性層
class ProjectionLayer(nn.Module):
    def __init__(self, d_model: int, vocab_size: int) -> None: # 模型維度和輸出詞彙表大小
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size) # 線性層，用於將 'd_model' 的特徵空間投影到 'vocab_size' 的輸出空間
        
    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim=-1) # 對輸出應用對數 Softmax 函數


In [ ]:
# 創建 Transformer 架構
class Transformer(nn.Module):
    
    # 這裡接收編碼器和解碼器，以及源語言和目標語言的嵌入。
    # 還接收源語言和目標語言的位置編碼，以及投影層
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer
        
    # 編碼器     
    def encode(self, src, src_mask):
        src = self.src_embed(src) # 將源語言嵌入應用於輸入的源語言
        src = self.src_pos(src) # 將源語言的位置編碼應用於源語言嵌入
        return self.encoder(src, src_mask) # 返回源語言嵌入和源語言掩碼，以防止注意某些元素
    
    # 解碼器
    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt) # 將目標語言嵌入應用於輸入的目標語言 (tgt)
        tgt = self.tgt_pos(tgt) # 將目標語言的位置編碼應用於目標語言嵌入
        
        # 返回目標語言嵌入、編碼器輸出以及源語言和目標語言的掩碼
        # 目標語言掩碼確保模型不會“看到”序列的未來元素
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    # 將投影層與 Softmax 函數應用於解碼器輸出
    def project(self, x):
        return self.projection_layer(x)


In [ ]:
# 創建並初始化 Transformer

# 定義函數及其參數，包括模型維度、編碼器和解碼器堆疊數量、頭數等
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512, N: int = 6, h: int = 8, dropout: float = 0.1, d_ff: int = 2048) -> Transformer:
    
    # 創建嵌入層
    src_embed = InputEmbeddings(d_model, src_vocab_size) # 源語言（從源詞彙表到 512 維向量）
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size) # 目標語言（從目標詞彙表到 512 維向量）
    
    # 創建位置編碼層
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout) # 源語言嵌入的位置編碼
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout) # 目標語言嵌入的位置編碼
    
    # 創建編碼器塊
    encoder_blocks = [] # 初始化空的編碼器塊列表
    for _ in range(N): # 迭代 'N' 次創建 'N' 個編碼器塊（N = 6）
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # 自注意力塊
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # 前饋神經網絡塊
        
        # 將層結合成編碼器塊
        encoder_block = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block) # 將編碼器塊添加到編碼器塊列表中
        
    # 創建解碼器塊
    decoder_blocks = [] # 初始化空的解碼器塊列表
    for _ in range(N): # 迭代 'N' 次創建 'N' 個解碼器塊（N = 6）
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # 自注意力塊
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout) # 交叉注意力塊
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout) # 前饋神經網絡塊
        
        # 將層結合成解碼器塊
        decoder_block = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block) # 將解碼器塊添加到解碼器塊列表中
        
    # 使用編碼器塊和解碼器塊列表創建編碼器和解碼器
    encoder = Encoder(nn.ModuleList(encoder_blocks))
    decoder = Decoder(nn.ModuleList(decoder_blocks))
    
    # 創建投影層
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size) # 將解碼器的輸出映射到目標詞彙空間
    
    # 通過結合上述所有部分創建 Transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    
    # 初始化參數
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
            
    return transformer # 組裝並初始化的 Transformer，準備進行訓練和驗證！


### Tokenizer

In [ ]:
# 定義分詞器
def build_tokenizer(config, ds, lang):
    
    # 創建分詞器的文件路徑
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    
    # 檢查分詞器是否已經存在
    if not Path.exists(tokenizer_path): 
        
        # 如果不存在，我們創建一個新的
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]')) # 初始化一個新的詞級分詞器
        tokenizer.pre_tokenizer = Whitespace() # 我們將根據空白字符將文本拆分為標記
        
        # 為新的分詞器創建訓練器
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", 
                                                   "[SOS]", "[EOS]"], min_frequency=2) # 定義詞級策略和特殊標記
        
        # 使用指定數據集和語言的句子訓練新的分詞器
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path)) # 將訓練好的分詞器保存到函數開始時指定的文件路徑
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path)) # 如果分詞器已經存在，我們加載它
    return tokenizer # 返回加載的分詞器或訓練好的分詞器


## Get Dataset

In [ ]:
def get_ds(config):
    
    # 加載 OpusBooks 數據集的訓練部分
    # 語言對將在稍後構建的 'config' 字典中定義
    ds_raw = load_dataset('opus_books', f'{config["lang_src"]}-{config["lang_tgt"]}', split='train') 
    
    # 為源語言和目標語言構建或加載分詞器
    tokenizer_src = build_tokenizer(config, ds_raw, config['lang_src'])
    tokenizer_tgt = build_tokenizer(config, ds_raw, config['lang_tgt'])
    
    # 將數據集劃分為訓練集和驗證集
    train_ds_size = int(0.9 * len(ds_raw)) # 90% 用於訓練
    val_ds_size = len(ds_raw) - train_ds_size # 10% 用於驗證
    train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size]) # 隨機劃分數據集
                                    
    # 使用 BilingualDataset 類處理數據，我們將在下面定義這個類
    train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
    val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
                                    
    # 遍歷整個數據集並打印源語言和目標語言句子中的最大長度
    max_len_src = 0
    max_len_tgt = 0
    for pair in ds_raw:
        src_ids = tokenizer_src.encode(pair['translation'][config['lang_src']]).ids
        tgt_ids = tokenizer_tgt.encode(pair['translation'][config['lang_tgt']]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))
        
    print(f'源語言句子的最大長度: {max_len_src}')
    print(f'目標語言句子的最大長度: {max_len_tgt}')
    
    # 為訓練集和驗證集創建數據加載器
    # 數據加載器用於在訓練和驗證過程中以批量方式迭代數據集
    train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True) # 批量大小將在配置字典中定義
    val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)
    
    return train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt # 返回 DataLoader 對象和分詞器


In [ ]:
def casual_mask(size):
    # 創建一個維度為 'size x size' 的方陣，全部填充為 1
    mask = torch.triu(torch.ones(1, size, size), diagonal=1).type(torch.int)
    return mask == 0


In [ ]:
class BilingualDataset(Dataset):
    
    # 接收包含句子對的數據集、源語言和目標語言的分詞器，以及源語言和目標語言的字符串
    # 'seq_len' 定義了兩種語言的序列長度
    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len) -> None:
        super().__init__()
        
        self.seq_len = seq_len
        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        
        # 使用目標語言分詞器定義特殊標記
        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

        
    # 數據集中實例的總數（某些句子對的長度不同）
    def __len__(self):
        return len(self.ds)
    
    # 使用索引檢索源語言和目標語言的文本
    def __getitem__(self, index: Any) -> Any:
        src_target_pair = self.ds[index]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]
        
        # 將源語言和目標語言的文本進行分詞
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids
        
        # 計算需要添加到分詞文本中的填充標記數量
        # 源語言標記
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2 # 減去兩個 '[EOS]' 和 '[SOS]' 特殊標記
        # 目標語言標記
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1 # 減去 '[SOS]' 特殊標記
        
        # 如果文本超過允許的 'seq_len'，則會引發錯誤。這意味著句子對中的某個句子太長，無法根據當前的序列長度限制進行處理
        # （這將在下面的配置字典中定義）
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError('句子太長')
         
        # 通過結合多個元素構建編碼器輸入張量
        encoder_input = torch.cat(
            [
            self.sos_token, # 插入 '[SOS]' 標記
            torch.tensor(enc_input_tokens, dtype=torch.int64), # 插入分詞後的源語言文本
            self.eos_token, # 插入 '[EOS]' 標記
            torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64) # 添加填充標記
            ]
        )
        
        # 通過結合多個元素構建解碼器輸入張量
        decoder_input = torch.cat(
            [
                self.sos_token, # 插入 '[SOS]' 標記 
                torch.tensor(dec_input_tokens, dtype=torch.int64), # 插入分詞後的目標語言文本
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64) # 添加填充標記
            ]
        
        )
        
        # 創建標籤張量，即模型訓練的預期輸出
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64), # 插入分詞後的目標語言文本
                self.eos_token, # 插入 '[EOS]' 標記 
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64) # 添加填充標記
                
            ]
        )
        
        # 確保上述每個張量的長度等於定義的 'seq_len'
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len
        
        return {
            'encoder_input': encoder_input,
            'decoder_input': decoder_input, 
            'encoder_mask': (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            'decoder_mask': (decoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int() & casual_mask(decoder_input.size(0)), 
            'label': label,
            'src_text': src_text,
            'tgt_text': tgt_text
        }


In [ ]:
# 定義函數以獲取最可能的下一個標記
def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    # 獲取目標標記序列的開始和結束標記的索引
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')
    
    # 計算源序列的編碼器輸出
    encoder_output = model.encode(source, source_mask)
    # 用開始標記初始化解碼器輸入
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    
    # 迭代直到達到最大長度 'max_len'
    while True:
        if decoder_input.size(1) == max_len:
            break
            
        # 為解碼器輸入構建掩碼
        decoder_mask = casual_mask(decoder_input.size(1)).type_as(source_mask).to(device)
        
        # 計算解碼器輸出
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        
        # 應用投影層以獲取下一個標記的概率
        prob = model.project(out[:, -1])
        
        # 選擇概率最高的標記
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
        
        # 如果下一個標記是結束標記，我們結束循環
        if next_word == eos_idx:
            break
            
    return decoder_input.squeeze(0) # 解碼器生成的標記序列


In [ ]:
# 定義函數以在驗證數據集上評估模型
# num_examples = 2，每次運行兩個示例
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_state, writer, num_examples=2):
    model.eval() # 將模型設置為評估模式
    count = 0 # 初始化計數器以跟蹤處理了多少示例
    
    console_width = 80 # 固定寬度的打印消息
    
    # 創建評估循環
    with torch.no_grad(): # 確保在此過程中不計算梯度
        for batch in validation_ds:
            count += 1
            encoder_input = batch['encoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            
            # 確保驗證集的 batch_size 為 1
            assert encoder_input.size(0) ==  1, '驗證集的批量大小必須為 1。'
            
            # 應用 'greedy_decode' 函數以獲取輸入批次的源文本的模型輸出
            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)
            
            # 從批次中檢索源語言和目標語言文本
            source_text = batch['src_text'][0]
            target_text = batch['tgt_text'][0] # 真實翻譯
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy()) # 解碼為人類可讀的模型輸出
            
            # 打印結果
            print_msg('-'*console_width)
            print_msg(f'源語言: {source_text}')
            print_msg(f'目標語言: {target_text}')
            print_msg(f'模型預測: {model_out_text}')
            
            # 處理兩個示例後，退出循環
            if count == num_examples:
                break


In [ ]:
# 我們將配置字典、源語言和目標語言的詞彙表長度作為參數傳遞
def get_model(config, vocab_src_len, vocab_tgt_len):
    
    # 使用 'build_transformer' 函數加載模型
    # 我們將使用源語言和目標語言詞彙表的長度、'seq_len' 和嵌入的維度
    model = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'], config['seq_len'], config['d_model'])
    return model


In [ ]:
# 定義構建和訓練 Transformer 模型的設置
def get_config():
    return {
        'batch_size': 8,
        'num_epochs': 20,
        'lr': 10**-4,
        'seq_len': 350,
        'd_model': 512, # Transformer 中嵌入的維度。512 和 "Attention Is All You Need" 論文中相同。
        'lang_src': 'en',
        'lang_tgt': 'it',
        'model_folder': 'weights',
        'model_basename': 'tmodel_',
        'preload': None,
        'tokenizer_file': 'tokenizer_{0}.json',
        'experiment_name': 'runs/tmodel'
    }
    

# 構建用於保存和檢索模型權重的路徑的函數
def get_weights_file_path(config, epoch: str):
    model_folder = config['model_folder'] # 從配置中提取模型文件夾
    model_basename = config['model_basename'] # 提取模型文件的基本名稱
    model_filename = f"{model_basename}{epoch}.pt" # 構建文件名
    return str(Path('.') / model_folder / model_filename) # 組合當前目錄、模型文件夾和模型文件名


In [ ]:
# 定義用於構建和訓練 Transformer 模型的設置
def train_model(config):
    # 設置設備以在 GPU 上運行，以加快訓練速度
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用設備 {device}")
    
    # 創建模型目錄以存儲權重
    Path(config['model_folder']).mkdir(parents=True, exist_ok=True)
    
    # 使用 'get_ds' 函數檢索源語言和目標語言的數據加載器和分詞器
    train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
    
    # 使用 'get_model' 函數初始化 GPU 上的模型
    model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
    
    # Tensorboard
    writer = SummaryWriter(config['experiment_name'])
    
    # 使用配置字典中的指定學習率和 epsilon 值設置 Adam 優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)
    
    # 初始化 epoch 和 global step 變量
    initial_epoch = 0
    global_step = 0
    
    # 檢查是否有預訓練模型要加載
    # 如果有，則加載它
    if config['preload']:
        model_filename = get_weights_file_path(config, config['preload'])
        print(f'預加載模型 {model_filename}')
        state = torch.load(model_filename) # 加載模型
        
        # 將 epoch 設置為保存的狀態加一，以從中斷處繼續
        initial_epoch = state['epoch'] + 1
        # 加載保存的優化器狀態
        optimizer.load_state_dict(state['optimizer_state_dict'])
        # 加載保存的全局步驟狀態
        global_step = state['global_step']
        
    # 初始化 CrossEntropyLoss 函數以進行訓練
    # 在計算損失時忽略填充標記，因為它們對學習過程無關緊要
    # 我們還應用標籤平滑以防止過擬合
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
    
    # 初始化訓練循環 
    # 從 'initial_epoch' 變量開始迭代到配置中指定的 epoch 數量
    for epoch in range(initial_epoch, config['num_epochs']):
        
        # 初始化訓練數據加載器的迭代器
        # 我們還使用 tqdm 顯示進度條
        batch_iterator = tqdm(train_dataloader, desc=f'Processing epoch {epoch:02d}')
        
        # 對每個批次...
        for batch in batch_iterator:
            model.train() # 訓練模型
            
            # 將輸入數據和掩碼加載到 GPU
            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            encoder_mask = batch['encoder_mask'].to(device)
            decoder_mask = batch['decoder_mask'].to(device)
            
            # 通過 Transformer 運行張量
            encoder_output = model.encode(encoder_input, encoder_mask)
            decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
            proj_output = model.project(decoder_output)
            
            # 將目標標籤加載到 GPU
            label = batch['label'].to(device)
            
            # 計算模型輸出和真實標籤之間的損失
            loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
            
            # 更新進度條
            batch_iterator.set_postfix({f"loss": f"{loss.item():6.3f}"})
            
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()
            
            # 進行反向傳播
            loss.backward()
            
            # 基於梯度更新參數
            optimizer.step()
            
            # 清除梯度以準備下一個批次
            optimizer.zero_grad()
            
            global_step += 1 # 更新全局步驟計數
            
        # 在每個 epoch 結束時運行 'run_validation' 函數以評估模型性能
        run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: batch_iterator.write(msg), global_step, writer)
         
        # 保存模型
        model_filename = get_weights_file_path(config, f'{epoch:02d}')
        # 將當前模型狀態寫入 'model_filename'
        torch.save({
            'epoch': epoch, # 當前 epoch
            'model_state_dict': model.state_dict(), # 當前模型狀態
            'optimizer_state_dict': optimizer.state_dict(), # 當前優化器狀態
            'global_step': global_step # 當前全局步驟
        }, model_filename)


In [ ]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore') # 過濾警告
    config = get_config() # 獲取配置設置
    train_model(config) # 使用配置參數訓練模型